In [1]:
import pandas as pd
import seaborn as sns
import xgboost
import matplotlib.pyplot as plt
import matplotlib
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#由于数据过大，我们选择先1000000个数据进行训练
train_data = pd.read_csv('./avazu-ctr-prediction/train.csv', nrows=1000000)
test_data = pd.read_csv('./avazu-ctr-prediction/test.csv')

In [3]:
print(train_data.info())
print(test_data.info())
#可以看到数据中没有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1000000 non-null  float64
 1   click             1000000 non-null  int64  
 2   hour              1000000 non-null  int64  
 3   C1                1000000 non-null  int64  
 4   banner_pos        1000000 non-null  int64  
 5   site_id           1000000 non-null  object 
 6   site_domain       1000000 non-null  object 
 7   site_category     1000000 non-null  object 
 8   app_id            1000000 non-null  object 
 9   app_domain        1000000 non-null  object 
 10  app_category      1000000 non-null  object 
 11  device_id         1000000 non-null  object 
 12  device_ip         1000000 non-null  object 
 13  device_model      1000000 non-null  object 
 14  device_type       1000000 non-null  int64  
 15  device_conn_type  1000000 non-null  int64  
 16  C

In [4]:
print(train_data.describe())
print(test_data.describe())

                 id           click          hour              C1  \
count  1.000000e+06  1000000.000000  1.000000e+06  1000000.000000   
mean   9.376314e+18        0.160219  1.410210e+07     1005.088166   
std    5.236908e+18        0.366809  1.493256e+00        1.156928   
min    9.984920e+12        0.000000  1.410210e+07     1001.000000   
25%    4.846660e+18        0.000000  1.410210e+07     1005.000000   
50%    9.834387e+18        0.000000  1.410210e+07     1005.000000   
75%    1.373053e+19        0.000000  1.410210e+07     1005.000000   
max    1.844670e+19        1.000000  1.410210e+07     1012.000000   

           banner_pos     device_type  device_conn_type             C14  \
count  1000000.000000  1000000.000000    1000000.000000  1000000.000000   
mean         0.229922        1.025540          0.223363    18262.203151   
std          0.464627        0.453899          0.667164     3510.366302   
min          0.000000        0.000000          0.000000      375.000000   
25%

<p>
id: 用户ID号
    
click: 0/1 表示未点击/点击

hour: 格式为YYMMDDHH，因此14091123表示2014年9月11日UTC时间23:00。
    
C1: 匿名分类变量
    
banner_pos: int型，网页上的广告位置，离散特征0,1,2,3...
    
site_id:Site ID
    
site_domain:Site领域
    
site_category: 网站类别
    
app_id: string型，用户APP的ID
    
app_domain
    
app_category

device_id: 设备编号

device_ip

device_model

device_type: 设备类型

device_conn_type:Device接入类型

C14-C21 -- anonymized categorical variables
</p>

In [5]:
#先尝试只用数值数字训练
df = train_data.loc[:,['click', 'hour', 'C1', 'banner_pos','device_type', 'device_conn_type', 'C14','C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']]
df_y = df['click']
df_x = df.drop(['click'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=5)

In [6]:
clf_xgb = xgboost.XGBClassifier(objective='binary:logistic', 
                                max_depth=5, 
                                learning_rate=0.1, 
                                n_estimators=128, 
                                min_child_weight=2,
                                seed=42)
clf_xgb.fit(X_train,
            y_train,
            eval_metric='auc',
            eval_set=[(X_test,y_test)])

d:\anaconda3\envs\pytorch\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.68513
[1]	validation_0-auc:0.68859
[2]	validation_0-auc:0.68856
[3]	validation_0-auc:0.68978
[4]	validation_0-auc:0.68989
[5]	validation_0-auc:0.69110
[6]	validation_0-auc:0.69171
[7]	validation_0-auc:0.69181
[8]	validation_0-auc:0.69818
[9]	validation_0-auc:0.69916
[10]	validation_0-auc:0.69991
[11]	validation_0-auc:0.69996
[12]	validation_0-auc:0.70088
[13]	validation_0-auc:0.70068
[14]	validation_0-auc:0.70164
[15]	validation_0-auc:0.70160
[16]	validation_0-auc:0.70150
[17]	validation_0-auc:0.70149
[18]	validation_0-auc:0.70174
[19]	validation_0-auc:0.70198
[20]	validation_0-auc:0.70276
[21]	validation_0-auc:0.70304
[22]	validation_0-auc:0.70323
[23]	validation_0-auc:0.70335
[24]	validation_0-auc:0.70471
[25]	validation_0-auc:0.70552
[26]	validation_0-auc:0.70593
[27]	validation_0-auc:0.70645
[28]	validation_0-auc:0.70706
[29]	validation_0-auc:0.70784
[30]	validation_0-auc:0.71000
[31]	validation_0-auc:0.71019
[32]	validation_0-auc:0.71062
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=128, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [ ]:
Id = pd.read_csv('./avazu-ctr-prediction/test.csv', usecols=[0], dtype=str)
df_test = test_data.loc[:,['hour', 'C1', 'banner_pos','device_type', 'device_conn_type', 'C14','C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']]
preds = clf_xgb.predict_proba(df_test)
output = pd.DataFrame({'id': Id.values.reshape(len(Id)),'click':preds[:,1:2].reshape(len(test_data))})
output.to_csv('./submission.csv', index=False)

<p>
单单如此我们竟然就在kaggle上取得了0.45503分的好成绩，虽然排名只有1411/1602，
    
但我们的分数与中上分段的差距并不大，而我们还有很大的提升空间，
    
    接下来我们可以优化的点主要有4个：
    
    1.对数值数据进行归一化等处理
    
    2.对非数值的数据用one-hot编码等方式数值化，加入训练模型中
    
    3.使用数据采样提取数据，而不是简单的使用前100000行数据
    
    4.调整模型参数，或者考虑使用LightGBM代替xgboost模型
</p>

In [2]:
#数据采样
#train.csv是按照时间排序的，只读取前100w条记录，会导致采样偏差
#我们将用户均匀分组，从每个分组去取合适数量的实例，以保证采取的样本尽可能对总体有代表性
#我们对label按比例取样，使得采样后click=1的数据占比不变
#我们按0.05的大小比例获取样本，减小数据集，方便后序操作
from sklearn.model_selection import StratifiedShuffleSplit
train_all = pd.read_csv('./avazu-ctr-prediction/train.csv')
split = StratifiedShuffleSplit(n_splits=1, train_size=0.05, random_state=42)
for train_index, test_index in split.split(train_all, train_all["click"]):
    strat_train_set = train_all.loc[train_index]
    strat_train_set.to_csv('./train_sample.csv', header=True, index=False)

In [2]:
#在采样后的数据上再尝试一次
train_data = pd.read_csv('./train_sample.csv')
df = train_data.loc[:,['click', 'hour', 'C1', 'banner_pos','device_type', 'device_conn_type', 'C14','C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']]
df_y = df['click']
df_x = df.drop(['click'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=5)
clf_xgb = xgboost.XGBClassifier(objective='binary:logistic', 
                                max_depth=5, 
                                learning_rate=0.1, 
                                n_estimators=256, 
                                min_child_weight=2,
                                seed=42)
clf_xgb.fit(X_train,
            y_train,
            eval_metric='auc',
            eval_set=[(X_test,y_test)])

d:\anaconda3\envs\pytorch\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.65128
[1]	validation_0-auc:0.65202
[2]	validation_0-auc:0.66172
[3]	validation_0-auc:0.66196
[4]	validation_0-auc:0.66197
[5]	validation_0-auc:0.66902
[6]	validation_0-auc:0.67126
[7]	validation_0-auc:0.67131
[8]	validation_0-auc:0.67333
[9]	validation_0-auc:0.67343
[10]	validation_0-auc:0.67439
[11]	validation_0-auc:0.67602
[12]	validation_0-auc:0.67643
[13]	validation_0-auc:0.67705
[14]	validation_0-auc:0.67731
[15]	validation_0-auc:0.67771
[16]	validation_0-auc:0.67825
[17]	validation_0-auc:0.67840
[18]	validation_0-auc:0.67876
[19]	validation_0-auc:0.67880
[20]	validation_0-auc:0.67965
[21]	validation_0-auc:0.68012
[22]	validation_0-auc:0.68037
[23]	validation_0-auc:0.68102
[24]	validation_0-auc:0.68131
[25]	validation_0-auc:0.68241
[26]	validation_0-auc:0.68253
[27]	validation_0-auc:0.68282
[28]	validation_0-auc:0.68317
[29]	validation_0-auc:0.68352
[30]	validation_0-auc:0.68402
[31]	validation_0-auc:0.68444
[32]	validation_0-auc:0.68500
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=256, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [6]:
test_data = pd.read_csv('./avazu-ctr-prediction/test.csv')
Id = pd.read_csv('./avazu-ctr-prediction/test.csv', usecols=[0], dtype=str)
df_test = test_data.loc[:,['hour', 'C1', 'banner_pos','device_type', 'device_conn_type', 'C14','C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']]
preds = clf_xgb.predict_proba(df_test)
output = pd.DataFrame({'id': Id.values.reshape(len(Id)),'click':preds[:,1:2].reshape(len(test_data))})
output.to_csv('./submission1.csv', index=False)

ValueError: cannot reshape array of size 2021448 into shape (4577464,)

<p>
可以看到在使用采样后的数据后，我们交叉验证的auc甚至降低了，表现不如从前

这可能是原来连续的数据间有很多的共性，我们推测这并不代表我们预测的准度下降了

这一猜想是正确的，事实上再提交后，我们在kaggle上的分数整整提高了0.03，已经达到了0.42006 排名1217/1602

可见取样的重要性
</p>

In [2]:
#拷贝object类型列
train_data = pd.read_csv('./train_sample.csv')
test_data = pd.read_csv('./avazu-ctr-prediction/test.csv')
df_y = train_data['click']
train_data.drop(['click'], axis=1, inplace=True)
Len = len(train_data)
df = pd.concat([train_data, test_data]).reset_index()
column_name = ['site_id','site_domain','site_category','app_id','app_domain','app_category','device_id','device_ip','device_model']
for itm in range(len(column_name)):
    category = df.loc[:,column_name[itm]]
    print('{}: {} \n num: {}'.format(column_name[itm], category.unique(), len(category.unique())))

site_id: ['4e7614cf' '43d6df75' '85f751fd' ... '4327e528' '3525557b' 'feb793b0'] 
 num: 3498
site_domain: ['c1aa3c04' '27e3c518' 'c4e18dd6' ... 'ae4ac9c7' 'bd3148f5' '94c52720'] 
 num: 4297
site_category: ['f028772b' '28905ebd' '50e219e0' '3e814130' '75fa27f6' 'f66779e6'
 '335d28a8' '76b2941d' 'c0dd3be3' 'dedf689d' '0569f928' '72722551'
 '70fb0e29' '42a36e14' '8fd0aea4' 'a818d37a' 'e787de0e' 'bcf865d9'
 'da34532e' 'c706e647' '5378d028' '9ccfa2ea' '74073276' 'a72a0145'] 
 num: 24
app_id: ['ecad2386' 'e2a1ca37' 'd36838b1' ... '95bf83c2' 'c66d1629' 'b47ce0ef'] 
 num: 5483
app_domain: ['7801e8d9' '2347f47a' '0e8616ad' 'aefc06bd' '5c5a694b' '33da2e74'
 '82e27996' 'ae637522' 'b9528b13' 'df32afa9' 'b8d325c3' 'd9b5648e'
 'ad63ec9b' '813f3323' '6f7ca2ba' '45a51db4' '5b9c592b' 'c6824def'
 '99b4c806' '885c7f3f' 'a8b0bf20' 'b5f3b24a' '47464e95' '5c620f04'
 '0654b444' '63f57be0' 'e5d5313f' '828da833' '51174fb1' '5ac0b939'
 '43cf4f06' '15ec7f39' 'b97def0d' '0d79ee56' 'b12ff13e' '063914ab'
 '6a0a3a9d

<p>
device_id的unique值非常之大，我们认为这缺少分类的意义

由于我们考虑使用one-hot编码进行数值化

两个category列不仅unique值很小，而且也和click有很大的现实联系

所以我们暂且只考虑这两列
</p>

In [3]:
df = pd.get_dummies(df, columns=['site_category'])
df = pd.get_dummies(df, columns=['app_category'])
df.drop(['id','site_id','site_domain','app_id','app_domain','device_id','device_ip','device_model'], axis=1, inplace=True)
df_x = df.iloc[0:Len, :]
test_data = df.iloc[Len:len(df), :]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=5)
clf_xgb = xgboost.XGBClassifier(objective='binary:logistic', 
                                max_depth=5, 
                                learning_rate=0.1, 
                                n_estimators=256, 
                                min_child_weight=2,
                                seed=42)
clf_xgb.fit(X_train,
            y_train,
            eval_metric='auc',
            eval_set=[(X_test,y_test)])

d:\anaconda3\envs\pytorch\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.67453
[1]	validation_0-auc:0.67498
[2]	validation_0-auc:0.68204
[3]	validation_0-auc:0.68400
[4]	validation_0-auc:0.68390
[5]	validation_0-auc:0.68534
[6]	validation_0-auc:0.68570
[7]	validation_0-auc:0.68836
[8]	validation_0-auc:0.68987
[9]	validation_0-auc:0.69078
[10]	validation_0-auc:0.69118
[11]	validation_0-auc:0.69166
[12]	validation_0-auc:0.69276
[13]	validation_0-auc:0.69303
[14]	validation_0-auc:0.69350
[15]	validation_0-auc:0.69456
[16]	validation_0-auc:0.69521
[17]	validation_0-auc:0.69578
[18]	validation_0-auc:0.69609
[19]	validation_0-auc:0.69623
[20]	validation_0-auc:0.69659
[21]	validation_0-auc:0.69667
[22]	validation_0-auc:0.69697
[23]	validation_0-auc:0.69781
[24]	validation_0-auc:0.69797
[25]	validation_0-auc:0.69837
[26]	validation_0-auc:0.69839
[27]	validation_0-auc:0.69863
[28]	validation_0-auc:0.69890
[29]	validation_0-auc:0.69889
[30]	validation_0-auc:0.69920
[31]	validation_0-auc:0.69936
[32]	validation_0-auc:0.69984
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=256, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [8]:
Id = pd.read_csv('./avazu-ctr-prediction/test.csv', usecols=[0], dtype=str)
preds = clf_xgb.predict_proba(test_data)
output = pd.DataFrame({'id': Id.values.reshape(len(Id)),'click':preds[:,1:2].reshape(len(test_data))})
output.to_csv('./submission3.csv', index=False)

<p>
使用one-hot编码增加特征后
    
我们的得分再次有了明显的提高
    
在kaggle的得分提升至0.40069
    
排名1073/1602
    
微调了xgb树高度限制 5->6 之后
    
我们再次提交没有啥变化

看来没啥用，我们再改回去
    
接下来是一些归一化处理
</p>

In [2]:
#归一化
train_data = pd.read_csv('./train_sample.csv')
test_data = pd.read_csv('./avazu-ctr-prediction/test.csv')
df_y = train_data['click']
train_data.drop(['click'], axis=1, inplace=True)
Len = len(train_data)
df = pd.concat([train_data, test_data]).reset_index()
column_name = ['site_id','site_domain','site_category','app_id','app_domain','app_category','device_id','device_ip','device_model']

In [3]:
scale = MinMaxScaler()
processing_columns = ['hour','C1','C14','C15','C16','C17','C18','C19','C20','C21']
processed_data = scale.fit_transform(df[processing_columns])

In [4]:
normalized_data = pd.DataFrame(processed_data, columns=['hour','C1','C14','C15','C16','C17','C18','C19','C20','C21'])
df.drop(processing_columns, axis=1, inplace=True)
df = pd.concat([df,normalized_data], axis=1)

In [6]:
df = pd.get_dummies(df, columns=['site_category'])
df = pd.get_dummies(df, columns=['app_category'])
df.drop(['id','site_id','site_domain','app_id','app_domain','device_id','device_ip','device_model'], axis=1, inplace=True)
df_x = df.iloc[0:Len, :]
test_data = df.iloc[Len:len(df), :]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=5)
clf_xgb = xgboost.XGBClassifier(objective='binary:logistic', 
                                max_depth=5, 
                                learning_rate=0.1, 
                                n_estimators=256, 
                                min_child_weight=2,
                                seed=42)
clf_xgb.fit(X_train,
            y_train,
            eval_metric='auc',
            eval_set=[(X_test,y_test)])

d:\anaconda3\envs\pytorch\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.66430
[1]	validation_0-auc:0.66777
[2]	validation_0-auc:0.67425
[3]	validation_0-auc:0.67600
[4]	validation_0-auc:0.67610
[5]	validation_0-auc:0.67661
[6]	validation_0-auc:0.67757
[7]	validation_0-auc:0.67945
[8]	validation_0-auc:0.68035
[9]	validation_0-auc:0.68192
[10]	validation_0-auc:0.68277
[11]	validation_0-auc:0.68324
[12]	validation_0-auc:0.68393
[13]	validation_0-auc:0.68456
[14]	validation_0-auc:0.68529
[15]	validation_0-auc:0.68629
[16]	validation_0-auc:0.68652
[17]	validation_0-auc:0.68649
[18]	validation_0-auc:0.68754
[19]	validation_0-auc:0.68751
[20]	validation_0-auc:0.68795
[21]	validation_0-auc:0.68841
[22]	validation_0-auc:0.68916
[23]	validation_0-auc:0.68925
[24]	validation_0-auc:0.68993
[25]	validation_0-auc:0.69020
[26]	validation_0-auc:0.69063
[27]	validation_0-auc:0.69102
[28]	validation_0-auc:0.69110
[29]	validation_0-auc:0.69133
[30]	validation_0-auc:0.69164
[31]	validation_0-auc:0.69216
[32]	validation_0-auc:0.69230
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=256, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [8]:
Id = pd.read_csv('./avazu-ctr-prediction/test.csv', usecols=[0], dtype=str)
preds = clf_xgb.predict_proba(test_data)
output = pd.DataFrame({'id': Id.values.reshape(len(Id)),'click':preds[:,1:2].reshape(len(test_data))})
output.to_csv('./submission4.csv', index=False)

<p>
归一化之后得分反而下降了
    
这并不令人惊讶
    
因为决策树并不是很需要归一化
    
往往会带来负收益
</p>

因此我们最终的得分是one-hot编码后未进行归一化的得分

既0.40069